# TokenTextSplitter
- LLM은 보통 토큰 제한이 있으므로 초과할 수 없다
- 텍스트를 토큰 수 기반으로 chunk를 생성할 때 사용한다.
## Tiktoken
- OpenAI에서 만든 `BPE Tokenizer`이다.

In [1]:
with open("./data/appendix-keywords.txt") as f:
    file = f.read()

**`CharaterTextSplitter.from_tiktoken_encoder`**
- `CharacterTextSplitter`에 의해서만 분할되고 `tiktoken` 토크나이저는 분할된 텍스트를 병합하는데 사용된다. 따라서 분할된 텍스트가 `tiktoken`으로 측정한 크기보다 클 수도 있다.  

**`RecursiveCharacterTextSplitter.from_tiktoken_encoder`**
- 분할된 텍스트가 LLM이 허용하는 토큰 크기보다 크지 않도록 할 수 있으며, 각 분할은 크기가 허용하는 크기보다 큰 경우 재귀적으로 분할된다.

In [2]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    # 청크 크기를 300으로 설정합니다.
    chunk_size=300,
    # 청크 간 중복되는 부분이 없도록 설정합니다.
    chunk_overlap=0,
)
# file 텍스트를 청크 단위로 분할합니다.
texts = text_splitter.split_text(file)

Created a chunk of size 358, which is longer than the specified 300
Created a chunk of size 315, which is longer than the specified 300
Created a chunk of size 305, which is longer than the specified 300
Created a chunk of size 366, which is longer than the specified 300
Created a chunk of size 330, which is longer than the specified 300
Created a chunk of size 351, which is longer than the specified 300
Created a chunk of size 378, which is longer than the specified 300
Created a chunk of size 361, which is longer than the specified 300
Created a chunk of size 350, which is longer than the specified 300
Created a chunk of size 362, which is longer than the specified 300
Created a chunk of size 335, which is longer than the specified 300
Created a chunk of size 353, which is longer than the specified 300
Created a chunk of size 358, which is longer than the specified 300
Created a chunk of size 336, which is longer than the specified 300
Created a chunk of size 324, which is longer tha

In [5]:
print(len(texts))
print(texts[0])

51
Semantic Search


In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4",
    chunk_size=100,
    chunk_overlap=0,
)

texts = text_splitter.split_text(file)

In [11]:
print(len(texts))
print(texts[0])

90
Semantic Search


# TokenTextSplitter
`tiktoken`을 직접 사용하여 각 분할이 LLM 허용 사이즈보다 작음을 보장할 수 있다

In [8]:
from langchain_text_splitters import TokenTextSplitter

text_splitter = TokenTextSplitter(
    chunk_size=100,  # 청크 크기를 10으로 설정합니다.
    chunk_overlap=0,  # 청크 간 중복을 0으로 설정합니다.
)

# state_of_the_union 텍스트를 청크로 분할합니다.
texts = text_splitter.split_text(file)
print(texts[0])  # 분할된 텍스트의 첫 번째 청크를 출력합니다.

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 �


# spaCy Splitter
- spaCy tokenizer로 텍스트를 분할할 수 있다 
- 문자 수로 chunk size를 측정한다.


In [12]:
!python -m spacy download en_core_web_sm --quiet

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [13]:
with open("./data/appendix-keywords.txt") as f:
    file = f.read()

In [14]:
import warnings
from langchain_text_splitters import SpacyTextSplitter

# 경고 메시지를 무시합니다.
warnings.filterwarnings("ignore")

# SpacyTextSplitter를 생성합니다.
text_splitter = SpacyTextSplitter(
    chunk_size=200,  # 청크 크기를 200으로 설정합니다.
    chunk_overlap=50,  # 청크 간 중복을 50으로 설정합니다.
)

In [15]:
texts = text_splitter.split_text(file)
print(texts[0])

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.


예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.


# SentenceTransformersTokenSplitter
- sentence-transformer 모델에 특화된 텍스트 분할기
- sentence transformer 모델의 토큰 윈도우에 맞게 텍스트를 청크로 분할

In [16]:
from langchain_text_splitters import SentenceTransformersTokenTextSplitter

# 문장 분할기를 생성하고 청크 간 중복을 0으로 설정합니다.
splitter = SentenceTransformersTokenTextSplitter(chunk_size=200, chunk_overlap=0)

In [17]:
count_start_and_stop_tokens = 2  # 시작과 종료 토큰의 개수를 2로 설정합니다.

# 텍스트의 토큰 개수에서 시작과 종료 토큰의 개수를 뺍니다.
text_token_count = splitter.count_tokens(
    text=file) - count_start_and_stop_tokens
print(text_token_count)  # 계산된 텍스트 토큰 개수를 출력합니다.

7686


In [18]:
text_chunks = splitter.split_text(text=file)

In [21]:
print(text_chunks[0])

semantic search 정의 : 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 [UNK] 결과를 반환하는 검색 방식입니다. [UNK] : 사용자가 " [UNK] 행성 " 이라고 검색하면, " 목성 ", " 화성 " 등과 [UNK] [UNK] 행성에 대한 정보를 반환합니다. 연관키워드 : 자연어 처리, 검색 알고리즘, 데이터 마이닝 embedding 정의 : 임베딩은 단어나 문장 [UNK] 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다


# NLTK
- NLTK Tokenizer 기반으로 텍스트 분할
- python에서 자연어 처리를 위한 라이브러리로, 텍스트 데이터 전처리, 형태소 분석, 품사 태깅, 토큰화 등의 기능이 있다.

In [22]:
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(
    chunk_size=200,  # 청크 크기를 200으로 설정합니다.
    chunk_overlap=0,  # 청크 간 중복을 0으로 설정합니다.
)

In [23]:
texts = text_splitter.split_text(file)
print(texts[0])  

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.

예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.


# Konlpy
- 한국어 자연어 처리를 위한 python 패키지
- 텍스트의 토큰 분할 효과는 언어 구조에 대한 토크나이저 이해에 크게 의존한다.
- 따라서 konlpy로 한국어에 좀 더 맞는 토크나이저로 분할하면 비교적 좋은 효과를 낼 수 있다

In [25]:
from langchain_text_splitters import KonlpyTextSplitter

# KonlpyTextSplitter를 사용하여 텍스트 분할기 객체를 생성합니다.
text_splitter = KonlpyTextSplitter()

In [27]:
texts = text_splitter.split_text(file)  # 한국어 문서를 문장 단위로 분할합니다.
print(len(texts))
print(texts[0])

2
Semantic Search 정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.

예시: 사용자가 " 태양계 행성" 이라고 검색하면, " 목성", " 화 성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.

연관 키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝 Embedding 정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다.

이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.

예시: " 사과" 라는 단어를 [0.65, -0.23, 0.17] 과 같은 벡터로 표현합니다.

연관 키워드: 자연어 처리, 벡터화, 딥 러닝 Token 정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다.

이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.

예시: 문장 " 나는 학교에 간다 "를 " 나는", " 학교에", " 간다" 로 분할합니다.

연관 키워드: 토큰 화, 자연어 처리, 구 문 분석 Tokenizer 정의: 토크 나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다.

이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.

예시: "I love programming." 이라는 문장을 [ "I", "love", "programming", "." ]으로 분할합니다.

연관 키워드: 토큰 화, 자연어 처리, 구 문 분석 VectorStore 정의: 벡터 스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템 입니

다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.

예시: 단어 임 베 딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다.

연관 키워드: 임 베 딩, 데이터베이스, 벡터화 SQL 정의: SQL(Structured Query Language) 은 데이터베이스에서 데이터를 관리하기 위한 프로그래밍 언어입니다.

데이터 조회, 수정, 삽입, 삭제 등 다양한 작업을 수

# HugginfaceTokenizer
- Huggingface에 있는 다양한 토크나이저를 활용할 수도 있다

In [28]:
from transformers import GPT2TokenizerFast
hf_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [34]:
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    # 허깅페이스 토크나이저를 사용하여 CharacterTextSplitter 객체를 생성합니다.
    hf_tokenizer,
    chunk_size=300,
    chunk_overlap=50,
)
# state_of_the_union 텍스트를 분할하여 texts 변수에 저장합니다.
texts = text_splitter.split_text(file)

Created a chunk of size 358, which is longer than the specified 300
Created a chunk of size 315, which is longer than the specified 300
Created a chunk of size 305, which is longer than the specified 300
Created a chunk of size 366, which is longer than the specified 300
Created a chunk of size 330, which is longer than the specified 300
Created a chunk of size 351, which is longer than the specified 300
Created a chunk of size 378, which is longer than the specified 300
Created a chunk of size 361, which is longer than the specified 300
Created a chunk of size 350, which is longer than the specified 300
Created a chunk of size 362, which is longer than the specified 300
Created a chunk of size 335, which is longer than the specified 300
Created a chunk of size 353, which is longer than the specified 300
Created a chunk of size 358, which is longer than the specified 300
Created a chunk of size 336, which is longer than the specified 300
Created a chunk of size 324, which is longer tha

In [35]:
print(len(texts))
print(texts[1])

51
정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝
